In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from lib.tools import *
from lib.jpeg import *
from lib.decomposition import *
from lib.regression import *
from lib.sketchml import *

## Generate a low rank tensor

In [2]:
tensor_shape = (128,128,3)
tensor_size  = 128*128*3
rank = 4

In [3]:
# CP decomposition: Creating factor matrices for the tensor
factor_1 = torch.randn(tensor_shape[0], rank)
factor_2 = torch.randn(tensor_shape[1], rank)
factor_3 = torch.randn(tensor_shape[2], rank)
tensor = torch.einsum('ir,jr,kr->ijk', factor_1, factor_2, factor_3)

In [4]:
# manipulate the tensor
value_mask = abs(tensor)>0.001
tensor = tensor* value_mask

## Get tensor charateristics

In [5]:
actual_rank= calculate_cp_rank(tensor.numpy())
print("The acutal rank of the tensor is: ", actual_rank)

Testing rank: 1
Error:0.8553295
Testing rank: 2
Error:0.33498165
Testing rank: 3
Error:0.11508464
Testing rank: 4
Error:9.266853e-09
The acutal rank of the tensor is:  4


In [6]:
sparsity = calculate_sparsity(tensor.numpy())
print("The sparsity of the tensor is: ", sparsity)

The sparsity of the tensor is:  0.001220703125


## Tensor compression

#### Jpeg

In [7]:
normalize_base, scale,zero_point, encoded_data, compressed_size_jpeg = compressor_jpeg(tensor, 100)
print("Compressed size: ", compressed_size_jpeg)

Compressed size:  42348


In [8]:
reconstructed_tensor_jpeg = decompressor_jpeg(tensor.shape,normalize_base, scale,zero_point, encoded_data )
snr_jpeg = calculate_snr(tensor_size,tensor.numpy(), reconstructed_tensor_jpeg.numpy())
mse_jpeg = calculate_mse(tensor.numpy(), reconstructed_tensor_jpeg.numpy())
print("Reconstruction snr (dB): ", snr_jpeg)
print("Reconstruction mse: ", mse_jpeg)

Reconstruction snr (dB):  27.381739616394043
Reconstruction mse:  0.002842217


#### Tensor decomposition

In [9]:
factors, compressed_size_decomposition = compressor_decomposition(tensor, actual_rank)
print("Compressed size: ", compressed_size_decomposition)

Compressed size:  4144


In [11]:
reconstructed_tensor_decomposition = decompressor_decomposition(factors )
snr_decomposition = calculate_snr(tensor_size,tensor.numpy(), reconstructed_tensor_decomposition)
mse_decompositionm = calculate_mse(tensor.numpy(), reconstructed_tensor_decomposition)
print("Reconstruction snr (dB): ", snr_decomposition)
print("Reconstruction mse: ", mse_decompositionm)

Reconstruction snr (dB):  96.10288619995117
Reconstruction mse:  3.8154244e-10


#### Regression

In [27]:
factors, x_pos, x_neg, compressed_size_regression = compressor_regression(tensor, 4)
print("Compressed size: ", compressed_size_regression)

Compressed size:  14336.0


In [28]:
reconstructed_tensor_regression = decompressor_regression(tensor.shape, factors, x_pos,x_neg )
snr_regression = calculate_snr(tensor_size,tensor.numpy(), reconstructed_tensor_regression.numpy())
mse_regression = calculate_mse(tensor.numpy(), reconstructed_tensor_regression.numpy())
print("Reconstruction snr (dB): ", snr_regression)
print("Reconstruction mse: ", mse_regression)

Reconstruction snr (dB):  3.909149169921875
Reconstruction mse:  0.6322898


#### Sketch ML

In [21]:
minmaxsketch,bucket_means, keys, compressed_size_sketchml = compressor_sketchml(tensor,128,0.5,3)
print("Compressed size: ", compressed_size_sketchml)

Compressed size:  66312


In [22]:
reconstructed_tensor_sketchml = decompressor_sketchml(tensor.shape, minmaxsketch,bucket_means, keys)
snr_sketchml = calculate_snr(tensor_size,tensor.numpy(), reconstructed_tensor_sketchml.numpy())
mse_sketchml = calculate_mse(tensor.numpy(), reconstructed_tensor_sketchml.numpy())
print("Reconstruction snr (dB): ", snr_sketchml)
print("Reconstruction mse: ", mse_sketchml)

Reconstruction snr (dB):  4.823217391967773
Reconstruction mse:  0.5122823
